In [1]:
import pandas as pd
import re
import string
import pickle
import numpy as np
regex = re.compile('[%s]' % re.escape(string.punctuation))
from collections import Counter

In [2]:
df = pd.read_csv('./processed_data.tsv', sep='\t',low_memory=False)


In [3]:
dialogues = df['Dialogue'].values.tolist()

In [4]:
vocab = Counter()
for line in dialogues:
    words = regex.sub('',line).lower().split()
    for w in words:
        vocab[w]+=1
len(vocab)



63107

In [5]:
min_count = 2
vocab_list=[]
for k,v in vocab.items():
    if v >= min_count:
        vocab_list.append(k)
len(vocab_list)

32728

In [6]:
input_seq=[]
output_seq=[]
for i,line in enumerate(dialogues):
    if i%2==0:
        input_seq.append(line)
    else:
        output_seq.append(line)
print(len(input_seq),'in')
print(len(output_seq),'out')


145618 in
145618 out


In [7]:
# for i in range(1,20):
#     print(input_seq[len(input_seq)-i],'+++',output_seq[len(input_seq)-i])

In [8]:
print(len(input_seq))
print(len(output_seq))

145618
145618


In [9]:
# for i in range(60,80):
#     print(input_seq[i],'+++',output_seq[i])

In [10]:
UNK_TOKEN = "<UNK>"
START_TOKEN = "<START>"
STOP_TOKEN = "<STOP>"

input_seq_reduced = []
for line in input_seq:
    input_seq_reduced.append(' '.join([UNK_TOKEN if t not in vocab_list else t for t in line.split()]))

In [11]:
output_seq_reduced = []
for line in output_seq:
    output_seq_reduced.append(' '.join([UNK_TOKEN if t not in vocab_list else t for t in line.split()]))

In [12]:
# for i in range(20):
#     print(input_seq_reduced[i],'+++',output_seq_reduced[i])

In [13]:
vocab_list+=[START_TOKEN,STOP_TOKEN]

In [14]:
word_to_index = {vocab_list[i]:i for i in range(len(vocab_list))}
index_to_word = {v: k for k, v in word_to_index.items()}

In [15]:
with open('glove_enc.pkl','rb') as handle:
    glove_dict=pickle.load(handle)

In [16]:
print(len(vocab_list))
len(glove_dict)

32730


28377

In [17]:
cross=[]
for w in vocab_list:
    if w not in glove_dict.keys():
        cross.append(w)
len(cross)

4353

In [18]:
glove_dict[UNK_TOKEN]=np.zeros(50,dtype='float')

In [19]:
ip_rows=[]
op_rows=[]
for i in range(len(input_seq_reduced)):
    ip_line = regex.sub('',input_seq_reduced[i].lower())
    op_line = regex.sub('',output_seq_reduced[i].lower())
    if len(ip_line.split()) <30 and len(op_line.split()) <30:
        ip_row=[START_TOKEN for _ in range(30-len(ip_line.split()))]+ip_line.split()+[STOP_TOKEN]
        ip_rows.append(ip_row)
        op_row=[START_TOKEN for _ in range(30-len(op_line.split()))]+op_line.split()+[STOP_TOKEN]
        op_rows.append(op_row)

In [20]:
len(input_seq_reduced)

145618

In [21]:
dim3array=[]
for se in ip_rows:
    dim2arr=[]
    for word in se:
        if word in glove_dict:
            dim2arr.append(glove_dict[word])
        else:
            dim2arr.append(np.zeros(50,dtype=float))
    dim3array.append(dim2arr)
dim3array=np.array(dim3array)

In [22]:
dim3array.shape

(130182, 31, 50)

In [23]:
dim3array_op=[]
for se in op_rows:
    dim2arr=[]
    for word in se:
        if word in glove_dict:
            dim2arr.append(glove_dict[word])
        else:
            dim2arr.append(np.zeros(50,dtype=float))
    dim3array_op.append(dim2arr)
dim3array_op=np.array(dim3array_op)
dim3array_op.shape

(130182, 31, 50)

In [24]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

C:\Users\abhil\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [36]:
batch_size = 64  # Batch size for training.
epochs = 1  # Number of epochs to train for.
latent_dim = 256

encoder_inputs = Input(shape=(None, 50))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, 50))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(50, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [37]:
model.fit([dim3array, dim3array_op[:,:-1,:]], dim3array_op[:,1:,:],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 104145 samples, validate on 26037 samples
Epoch 1/1
  5120/104145 [>.............................] - ETA: 4:34 - loss: -11.5683

KeyboardInterrupt: 

In [32]:
a=[[[1,2,3],[4,5,6]]]
a=np.array(a)
a.shape
a[:,1:,:]

array([[[4, 5, 6]]])